In [1]:
import pandas as pd

df = pd.read_csv('./task2.csv')

In [2]:
import os
import json

# Cargar el CSV
df = pd.read_csv('./task2.csv')

# Primero, veamos qué hay en el DataFrame
print("Columnas en el CSV:", df.columns.tolist())
print("\nPrimeras filas del DataFrame:")
print(df.head())

# Extraer los IDs únicos de la carpeta 2018
path = './task2/2018'
json_files = [f for f in os.listdir(path) if f.endswith('EN.json')]
print(f"\nArchivos JSON encontrados: {len(json_files)}")

drafts = []
for json_file in json_files:
    try:
        with open(os.path.join(path, json_file)) as f:
            data = json.load(f)
            drafts.append(data['Content'])
            print(f"✓ Procesado: {json_file}")
    except Exception as e:
        print(f"✗ Error procesando {json_file}: {str(e)}")

print(f"\nTotal de documentos procesados: {len(drafts)}")

# Si se procesaron documentos, mostrar ejemplo del contenido
if drafts:
    print("\nEjemplo del primer documento (primeros 200 caracteres):")
    print(drafts[0][:200])

Columnas en el CSV: ['Country', 'Original_id', 'Authors', 'Voting', 'Date']

Primeras filas del DataFrame:
          Country  Original_id  \
0  United Kingdom           72   
1          France           73   
2          Russia          322   
3   United States          341   
4   United States          455   

                                             Authors Voting        Date  
0  ['Bolivia (Plurinational State of)', 'China', ...      Y  2018-01-30  
1  ['Bolivia (Plurinational State of)', 'China', ...      Y  2018-01-30  
2                                                NaN      Y   13/4/2018  
3  ['Bolivia (Plurinational State of)', 'China', ...      Y   23/4/2018  
4  ['Bolivia (Plurinational State of)', 'China', ...      Y   15/5/2018  

Archivos JSON encontrados: 10
✓ Procesado: S_2018_1147-EN.json
✓ Procesado: S_2018_455-EN.json
✓ Procesado: S_2018_72-EN.json
✓ Procesado: S_2018_1110-EN.json
✓ Procesado: S_2018_1109-EN.json
✓ Procesado: S_2018_73-EN.json
✓ Procesado: S_2018_

In [3]:
# If use Together API
from together import Together

your_model_name = 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free'
your_api_key = '574bf95f6b33074e60f284c2c6316e72c5ef8226ce8ce4650dacca4d51ce1050'
client = Together(api_key=your_api_key)

In [4]:
import random
from tqdm import tqdm
import pandas as pd
import os
import json
from together import Together
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import roc_auc_score, average_precision_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder, label_binarize
from imblearn.metrics import geometric_mean_score
import numpy as np

# Definir la función calculate_metrics primero
def calculate_metrics(pred, labels):
    label_encoder = LabelEncoder()
    all_classes = list(set(labels) | set(pred))
    label_encoder.fit(all_classes)

    labels = label_encoder.transform(labels)
    pred = label_encoder.transform(pred)

    acc = accuracy_score(labels, pred)
    
    num_classes = len(label_encoder.classes_)
    true_labels_bin = label_binarize(labels, classes=list(range(num_classes)))
    pred_bin = label_binarize(pred, classes=list(range(num_classes)))

    auc = roc_auc_score(true_labels_bin, pred_bin, multi_class='ovr', average='macro')
    pr_auc = average_precision_score(true_labels_bin, pred_bin, average='macro')

    balanced_acc = balanced_accuracy_score(labels, pred)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, pred, average='macro')

    mcc = matthews_corrcoef(labels, pred)
    g_mean = geometric_mean_score(labels, pred, average='macro')

    print(f'Accuracy: {acc}')
    print(f'AUC: {auc}')
    print(f'Balanced Accuracy: {balanced_acc}')
    print(f'Precision: {prec}')
    print(f'Recall: {rec}')
    print(f'F1: {f1}')
    print(f'PR AUC: {pr_auc}')
    print(f'MCC: {mcc}')
    print(f'G-Mean: {g_mean}')

    print('Accuracy AUC Balanced_Acc Precision Recall F1 PR_AUC MCC G-Mean')
    print(f'{acc:.4f} {auc:.4f} {balanced_acc:.4f} {prec:.4f} {rec:.4f} {f1:.4f} {pr_auc:.4f} {mcc:.4f} {g_mean:.4f}')

# Inicializar todas las variables necesarias
drafts = []
pred = []
invalid_responses = []
votes = []

# Cargar el CSV y obtener los votos
df = pd.read_csv('./task2.csv')
votes = list(df['Voting'])

# Configurar Together API
your_model_name = 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free'
your_api_key = '574bf95f6b33074e60f284c2c6316e72c5ef8226ce8ce4650dacca4d51ce1050'
client = Together(api_key=your_api_key)

# Mostrar información del DataFrame
print("Columnas en el CSV:", df.columns.tolist())
print("\nPrimeras filas del DataFrame:")
print(df.head())

# Procesar archivos JSON
path = './task2/2018'
json_files = [f for f in os.listdir(path) if f.endswith('EN.json')]
print(f"\nArchivos JSON encontrados: {len(json_files)}")

# Cargar contenido de los archivos JSON
for json_file in json_files:
    try:
        with open(os.path.join(path, json_file)) as f:
            data = json.load(f)
            drafts.append(data['Content'])
            print(f"✓ Procesado: {json_file}")
    except Exception as e:
        print(f"✗ Error procesando {json_file}: {str(e)}")

print(f"\nTotal de documentos procesados: {len(drafts)}")

def truncate_text(text, max_chars=2000):
    if len(text) > max_chars:
        return text[:max_chars] + "... [Texto truncado]"
    return text

# Procesar documentos con el modelo
print("\nProcesando documentos con el modelo...")
for i, draft in enumerate(drafts):
    try:
        truncated_draft = truncate_text(draft)
        country = df.iloc[i]['Country'] if i < len(df) else "Unknown"
        
        system_prompt = "You are a UN diplomat. Vote Y (Yes), N (No), or A (Abstain)."
        user_prompt = f"""As {country}'s diplomat, vote on this UN resolution:

{truncated_draft}

Vote (Y/N/A):"""
        
        response = client.chat.completions.create(
            model=your_model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=1,
            temperature=0.0
        )
        
        result = response.choices[0].message.content.strip()
        valid_votes = ['Y', 'N', 'A']
        
        if result not in valid_votes:
            print(f"\nRespuesta inválida para {country}: {result}")
            result = random.choice(valid_votes)
            invalid_responses.append(i)
        
        pred.append(result)
        print(f"País: {country}, Voto: {result}")
        
    except Exception as e:
        print(f"\nError procesando documento {i}: {str(e)}")
        result = random.choice(['Y', 'N', 'A'])
        invalid_responses.append(i)
        pred.append(result)

# Mostrar ejemplo del primer documento
if drafts:
    print("\nEjemplo del primer documento (primeros 200 caracteres):")
    print(drafts[0][:200])

# Mostrar respuestas inválidas
if len(invalid_responses) > 0:
    print("\nÍndices con respuestas inválidas:", invalid_responses)

# Calcular métricas usando la función ya definida
if len(pred) > 0 and len(votes) > 0:
    calculate_metrics(pred, votes)

# Mostrar respuestas inválidas
if len(invalid_responses) > 0:
    print("\nÍndices con respuestas inválidas:", invalid_responses)

# Calcular métricas
if len(pred) > 0 and len(votes) > 0:
    calculate_metrics(pred, votes)
# Si contamos con los votos reales, calculamos la precisión
if 'votes' in locals():
    correct = sum(1 for p, v in zip(pred, votes) if p == v)
    accuracy = correct / len(votes)
    print(f"\nPrecisión: {accuracy:.2%}")

Columnas en el CSV: ['Country', 'Original_id', 'Authors', 'Voting', 'Date']

Primeras filas del DataFrame:
          Country  Original_id  \
0  United Kingdom           72   
1          France           73   
2          Russia          322   
3   United States          341   
4   United States          455   

                                             Authors Voting        Date  
0  ['Bolivia (Plurinational State of)', 'China', ...      Y  2018-01-30  
1  ['Bolivia (Plurinational State of)', 'China', ...      Y  2018-01-30  
2                                                NaN      Y   13/4/2018  
3  ['Bolivia (Plurinational State of)', 'China', ...      Y   23/4/2018  
4  ['Bolivia (Plurinational State of)', 'China', ...      Y   15/5/2018  

Archivos JSON encontrados: 10
✓ Procesado: S_2018_1147-EN.json
✓ Procesado: S_2018_455-EN.json
✓ Procesado: S_2018_72-EN.json
✓ Procesado: S_2018_1110-EN.json
✓ Procesado: S_2018_1109-EN.json
✓ Procesado: S_2018_73-EN.json
✓ Procesado: S_2018_

/Users/deca3/UNBench/venv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/deca3/UNBench/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/deca3/UNBench/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/deca3/UNBench/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Specificity is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _war

In [7]:
# calculate metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import roc_auc_score, average_precision_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder, label_binarize
from imblearn.metrics import geometric_mean_score
import numpy as np

# calculate metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import roc_auc_score, average_precision_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder, label_binarize
from imblearn.metrics import geometric_mean_score
import numpy as np
import warnings

def calculate_metrics(pred, labels):
    # Suprimir todos los warnings
    warnings.filterwarnings('ignore')
    
    # Convertir a arrays numpy
    pred = np.array(pred)
    labels = np.array(labels)
    
    # Obtener clases únicas presentes en ambos conjuntos
    unique_classes = np.unique(np.concatenate([pred, labels]))
    
    # Métricas básicas
    acc = accuracy_score(labels, pred)
    balanced_acc = balanced_accuracy_score(labels, pred)
    
    # Calcular precision, recall y F1 con manejo de zero_division
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, 
        pred, 
        average='macro', 
        zero_division=1,  # Cambiar a 1 para evitar warnings
        labels=unique_classes
    )
    
    # Calcular MCC
    mcc = matthews_corrcoef(labels, pred)
    
    # Calcular G-Mean con manejo de errores
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        g_mean = geometric_mean_score(labels, pred, average='macro')

    # Imprimir resultados básicos
    print("\nMétricas básicas:")
    print(f'Accuracy: {acc:.4f}')
    print(f'Balanced Accuracy: {balanced_acc:.4f}')
    print(f'Precision: {prec:.4f}')
    print(f'Recall: {rec:.4f}')
    print(f'F1: {f1:.4f}')
    print(f'MCC: {mcc:.4f}')
    print(f'G-Mean: {g_mean:.4f}')
    
    # Calcular AUC y PR-AUC solo si hay múltiples clases
    if len(unique_classes) > 1:
        try:
            # Preparar datos para métricas binarias/multiclase
            label_encoder = LabelEncoder()
            label_encoder.fit(unique_classes)
            y_true_encoded = label_encoder.transform(labels)
            y_pred_encoded = label_encoder.transform(pred)
            
            # Binarizar las etiquetas
            y_true_bin = label_binarize(y_true_encoded, classes=range(len(unique_classes)))
            y_pred_bin = label_binarize(y_pred_encoded, classes=range(len(unique_classes)))
            
            # Calcular AUC y PR-AUC
            auc = roc_auc_score(y_true_bin, y_pred_bin, multi_class='ovr', average='macro')
            pr_auc = average_precision_score(y_true_bin, y_pred_bin, average='macro')
            
            print('\nMétricas adicionales:')
            print(f'AUC: {auc:.4f}')
            print(f'PR AUC: {pr_auc:.4f}')
        except Exception as e:
            print('\nMétricas adicionales:')
            print('AUC: N/A')
            print('PR AUC: N/A')
            auc = float('nan')
            pr_auc = float('nan')
    else:
        print('\nMétricas adicionales:')
        print('AUC: N/A (solo una clase presente)')
        print('PR AUC: N/A (solo una clase presente)')
        auc = float('nan')
        pr_auc = float('nan')
    
    # Imprimir resumen final
    print('\nResumen de métricas:')
    print('Accuracy Balanced_Acc Precision Recall F1 MCC G-Mean')
    print(f'{acc:.4f} {balanced_acc:.4f} {prec:.4f} {rec:.4f} {f1:.4f} {mcc:.4f} {g_mean:.4f}')

    return {
        'accuracy': acc,
        'balanced_accuracy': balanced_acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'mcc': mcc,
        'g_mean': g_mean,
        'auc': auc,
        'pr_auc': pr_auc
    }

   

In [8]:
calculate_metrics(pred, votes)


Métricas básicas:
Accuracy: 0.8000
Balanced Accuracy: 0.8000
Precision: 0.5000
Recall: 0.9000
F1: 0.4444
MCC: 0.0000
G-Mean: 0.4000

Métricas adicionales:
AUC: nan
PR AUC: 1.0000

Resumen de métricas:
Accuracy Balanced_Acc Precision Recall F1 MCC G-Mean
0.8000 0.8000 0.5000 0.9000 0.4444 0.0000 0.4000


{'accuracy': 0.8,
 'balanced_accuracy': np.float64(0.8),
 'precision': 0.5,
 'recall': 0.9,
 'f1': 0.4444444444444444,
 'mcc': 0.0,
 'g_mean': np.float64(0.4),
 'auc': nan,
 'pr_auc': np.float64(1.0)}